In [ ]:
import glob
import os
import pathlib
import parse_access_cm2_results as par
import plot_pandas_data as ppd
import plot_right_axis_time as pra
from convert_time import days_to_gregorian_years, gregorian_years_to_days
from numpy import linspace

In [ ]:
um_atmos_archive_dir = pathlib.PurePath("../access-cm2/cm2_test/archive/")
work_dir_glob = os.path.join(um_atmos_archive_dir, "work.*.bench")
work_dir_list = glob.glob(work_dir_glob)

In [ ]:
df = par.parse_all(work_dir_list)

In [ ]:
bw = lambda d: d[d["npes_per_node"] % 7 == 0]
cl = lambda d: d[d["npes_per_node"] % 6 == 0]

In [ ]:
full = lambda d: d[d["npes_per_node"] >= 28]
half = lambda d: d[(d["npes_per_node"] == 14) | (d["npes_per_node"]) == 24]
qrtr = lambda d: d[(d["npes_per_node"] == 7) | (d["npes_per_node"]) == 12]

In [ ]:
atm1 = lambda d: d[d["atm_cols"] != 28]
atm2 = lambda d: d[d["atm_cols"] == 28]

In [ ]:
ocn0 = lambda d: d[d["ocn_npes"] == 64]
ocn1 = lambda d: d[d["ocn_npes"] == 128]
ocn2 = lambda d: d[d["ocn_npes"] == 256]

In [ ]:
omp1 = lambda d: d[d["omp_num_threads"] == 1]
omp2 = lambda d: d[d["omp_num_threads"] == 2]

In [ ]:
print(df.shape)

In [ ]:
xticks_ncpus = lambda d: d["ncpus"].sort_values().unique()
xticks_npes  = lambda d: d["atm_npes"].sort_values().unique()

In [ ]:
data=cl(ocn1(omp1(full(df))))

In [ ]:
ppd.loglog_by_group("ncpus","walltime",
    by=["atm_cols"], 
    data=data,
    xlabel="CM2 cores",
    ylabel="Walltime (s)",
    xticks=xticks_ncpus(data),               
    right_axis_fn=pra.no_right_axis,
    legend_title="UM NPROCY")

In [ ]:
ppd.semilogx_by_group("ncpus","walltime * ncpus",
    by=["atm_cols"], 
    data=data,
    xlabel="CM2 cores",
    ylabel="Walltime (s) x CM2 cores",
    xticks=xticks_ncpus(data),
    yticks=linspace(0,800000,5),
    right_axis_fn=pra.no_right_axis,
    legend_title="UM NPROCY")

In [ ]:
ppd.semilogx_by_group("ncpus","Atm_Step_4A (AS) * ncpus",
    by=["atm_cols"], 
    data=data,
    xlabel="CM2 cores",
    ylabel="UM time (s) x CM2 cores",
    xticks=xticks_ncpus(data),
    yticks=linspace(0,800000,5),
    right_axis_fn=pra.no_right_axis,
    legend_title="UM NPROCY")

In [ ]:
ppd.loglog_by_group("ncpus","Atm_Step_4A (AS) speed",
    by=["atm_cols"], 
    data=data,
    xlabel="CM2 cores",
    ylabel="UM speed (d/d)",
    xticks=xticks_ncpus(data),               
    right_axis_fn=pra.plot_right_axis_yd,
    legend_title="UM NPROCY")

In [ ]:
ppd.semilogx_by_group("ncpus","Atm_Step_4A (AS) * ncpus speed",
    by=["atm_cols"], 
    data=data,
    xlabel="CM2 cores",
    ylabel="UM speed (d/d) per CM2 core",
    xticks=xticks_ncpus(data),
    right_yticks=[0.002,0.005,0.01,0.02],
    right_axis_fn=pra.plot_right_axis_yd,
    legend_title="UM NPROCY")

In [ ]:
def print_speed(speeds):
    print("CPUs d/d  Y/d")
    for ncpus in speeds:
        speed = speeds[ncpus]
        print(
            f"{ncpus:-4}",
            round(speed,2),
            round(days_to_gregorian_years(speed),5))

In [ ]:
def print_speed_per_cpu(speeds_per_cpu):
    print("CPUs d/d  Y/d")
    for ncpus in speeds_per_cpu:
        speed = speeds_per_cpu[ncpus]
        print(
            f"{ncpus:-4}",
            round(speed,2),
            round(days_to_gregorian_years(speed),5))

In [ ]:
def derive_mean_speed(
    df, 
    by="atm_cols",
    speed="Atm_Step_4A (AS) speed"):    
    mean_speed = dict()
    group_cols = df.groupby(by)
    for name, group in group_cols:
        group_by_ncpus = group.groupby("ncpus")
        ncpus_list = group_by_ncpus.groups.keys()
        mean_by_ncpus = group_by_ncpus.mean()
        mean_speeds = mean_by_ncpus[speed]
        mean_speed[name] = {
           ncpus: mean_speeds[ncpus] for ncpus in ncpus_list}
    return mean_speed

In [ ]:
def derive_mean_speed_per_cpu(
    df, 
    by="atm_cols", 
    speed="Atm_Step_4A (AS) speed"):
    mean_speed_per_cpu = dict()
    group_cols = df.groupby(by)
    for name, group in group_cols:
        group_by_ncpus = group.groupby("ncpus")
        ncpus_list = group_by_ncpus.groups.keys()
        mean_by_ncpus = group_by_ncpus.mean()
        mean_speeds = mean_by_ncpus[speed]
        mean_speed_per_cpu[name] = {
           ncpus: mean_speeds[ncpus]/ncpus for ncpus in ncpus_list}
    return mean_speed_per_cpu

In [ ]:
mean_speed_per_cpu = derive_mean_speed_per_cpu(
    data,
    by="atm_cols")
    
print("Mean speed per CM2 core for UM NPROCY: 12")
print_speed_per_cpu(mean_speed_per_cpu[12])
print("Mean speed per CM2 core for UM NPROCY: 24")
print_speed_per_cpu(mean_speed_per_cpu[24])
print("Mean speed per CM2 core for UM NPROCY: 28")
print_speed_per_cpu(mean_speed_per_cpu[28])

speed_24x12_vs_12x24 = round(
    mean_speed_per_cpu[12][432]/mean_speed_per_cpu[24][432],3)
speed_28x24_vs_24x28=round(
    mean_speed_per_cpu[24][816]/mean_speed_per_cpu[28][816],3)

print("Speed at 24 x 12 vs 12 x 24")
print(speed_24x12_vs_12x24)
print("Speed at 28 x 24 vs 24 x 28")
print(speed_28x24_vs_24x28)

In [ ]:
data=cl(ocn1(atm1(df)))

In [ ]:
ppd.loglog_by_group("ncpus","walltime",
    by=["omp_num_threads","atm_cols"], 
    data=data,
    xlabel="CM2 cores",
    ylabel="Walltime (s)",
    xticks=xticks_ncpus(data),               
    right_axis_fn=pra.no_right_axis,
    legend_title="OMP_NUM_THREADS,UM NPROCY")

In [ ]:
ppd.semilogx_by_group("ncpus","walltime * ncpus",
    by=["omp_num_threads","atm_cols"], 
    data=data,
    xlabel="CM2 cores",
    ylabel="Walltime (s) x CM2 cores",
    xticks=xticks_ncpus(data),
    yticks=linspace(0,1500000,4),
    right_axis_fn=pra.no_right_axis,
    legend_title="OMP_NUM_THREADS,UM NPROCY")

In [ ]:
ppd.semilogx_by_group("ncpus","Atm_Step_4A (AS) * ncpus",
    by=["omp_num_threads","atm_cols"], 
    data=data,
    xlabel="CM2 cores",
    ylabel="UM time (s) x CM2 cores",
    xticks=xticks_ncpus(data),
    yticks=linspace(0,1200000,5),
    right_axis_fn=pra.no_right_axis,
    legend_title="OMP_NUM_THREADS,UM NPROCY")

In [ ]:
ppd.loglog_by_group("ncpus","Atm_Step_4A (AS) speed",
    by=["omp_num_threads","atm_cols"], 
    data=data,
    xlabel="CM2 cores",
    ylabel="UM speed (d/d)",
    xticks=xticks_ncpus(data),             
    right_yticks=[0.5,1,2,5,10],
    right_axis_fn=pra.plot_right_axis_yd,
    legend_title="OMP_NUM_THREADS,\nUM NPROCY")

In [ ]:
ppd.semilogx_by_group("ncpus","Atm_Step_4A (AS) * ncpus speed",
    by=["omp_num_threads","atm_cols"], 
    data=data,
    xlabel="CM2 cores",
    ylabel="UM speed (d/d) per CM2 core",
    xticks=xticks_ncpus(data),               
    right_yticks=linspace(0,0.01,6),
    right_axis_fn=pra.plot_right_axis_yd,
    legend_title="OMP_NUM_THREADS,UM NPROCY")

In [ ]:
mean_speed_per_cpu = derive_mean_speed_per_cpu(
    data,
    by=["omp_num_threads","atm_cols"])
print("Mean speed per CM2 core for OMP_NUM_THREADS: 1, UM NPROCY: 12")
print_speed_per_cpu(mean_speed_per_cpu[(1,12)])
print("Mean speed per CM2 core for OMP_NUM_THREADS: 1, UM NPROCY: 24")
print_speed_per_cpu(mean_speed_per_cpu[(1,24)])
print("Mean speed per CM2 core for OMP_NUM_THREADS: 2, UM NPROCY: 12")
print_speed_per_cpu(mean_speed_per_cpu[(2,12)])
print("Mean speed per CM2 core for OMP_NUM_THREADS: 2, UM NPROCY: 24")
print_speed_per_cpu(mean_speed_per_cpu[(2,24)])

speed_omp1_vs_omp2 = dict()
speed_omp1_vs_omp2[432] = round(
    mean_speed_per_cpu[(1,12)][432]/mean_speed_per_cpu[(2,12)][432],2)
for ncpus in [720, 1296]:
    speed_omp1_vs_omp2[ncpus] = round(
        mean_speed_per_cpu[(1,24)][ncpus]/mean_speed_per_cpu[(2,24)][ncpus],2)
for ncpus in [432, 720, 1296]:
    print(f"Speed for 1 thread vs 2 threads for {ncpus} cores")
    print(speed_omp1_vs_omp2[ncpus])

In [ ]:
data=cl(omp1(atm1(full(df))))

In [ ]:
ppd.loglog_by_group("ncpus","walltime",
    by=["ocn_npes","atm_cols"], 
    data=data,
    xlabel="CM2 cores",
    ylabel="Walltime (s)",
    xticks=xticks_ncpus(data),               
    right_axis_fn=pra.no_right_axis,
    legend_title="MOM cores,UM NPROCY")

In [ ]:
ppd.semilogx_by_group("ncpus","walltime * ncpus",
    by=["ocn_npes","atm_cols"], 
    data=data,
    xlabel="CM2 cores",
    ylabel="Walltime (s) x CM2 cores",
    xticks=xticks_ncpus(data),
    yticks=linspace(0,800000,5),
    right_axis_fn=pra.no_right_axis,
    legend_title="MOM cores,UM NPROCY")

In [ ]:
ppd.semilogx_by_group("ncpus","Atm_Step_4A (AS) * ncpus",
    by=["ocn_npes","atm_cols"], 
    data=data,
    xlabel="CM2 cores",
    ylabel="UM time (s) x CM2 cores",
    xticks=xticks_ncpus(data),
    yticks=linspace(0,800000,5),
    right_axis_fn=pra.no_right_axis,
    legend_title="MOM cores,UM NPROCY")

In [ ]:
ppd.loglog_by_group("ncpus","Atm_Step_4A (AS) speed",
    by=["ocn_npes","atm_cols"], 
    data=data,
    xlabel="CM2 cores",
    ylabel="UM speed (d/d)",
    xticks=xticks_ncpus(data),
    right_axis_fn=pra.plot_right_axis_yd,
    legend_title="MOM cores,UM NPROCY")

In [ ]:
ppd.semilogx_by_group("ncpus","Atm_Step_4A (AS) * ncpus speed",
    by=["ocn_npes","atm_cols"], 
    data=data,
    xlabel="CM2 cores",
    ylabel="UM speed (d/d) per CM2 core",
    xticks=xticks_ncpus(data),               
    right_yticks=linspace(0,0.01,6),
    right_axis_fn=pra.plot_right_axis_yd,
    legend_title="MOM cores,UM NPROCY")

In [ ]:
ppd.loglog_by_group("atm_npes","walltime",
    by=["ocn_npes","atm_cols"], 
    data=data,
    xlabel="UM ranks",
    ylabel="Walltime (s)",
    xticks=xticks_npes(data),               
    right_axis_fn=pra.no_right_axis,
    legend_title="MOM cores,UM NPROCY")

In [ ]:
ppd.semilogx_by_group("atm_npes","walltime * ncpus",
    by=["ocn_npes","atm_cols"], 
    data=data,
    xlabel="UM ranks",
    ylabel="Walltime (s) x CM2 cores",
    xticks=xticks_npes(data),
    yticks=linspace(0,800000,5),
    right_axis_fn=pra.no_right_axis,
    legend_title="MOM cores,UM NPROCY")

In [ ]:
ppd.semilogx_by_group("atm_npes","Atm_Step_4A (AS) * ncpus",
    by=["ocn_npes","atm_cols"], 
    data=data,
    xlabel="UM ranks",
    ylabel="UM time (s) x CM2 cores",
    xticks=xticks_npes(data),
    yticks=linspace(0,800000,5),
    right_axis_fn=pra.no_right_axis,
    legend_title="MOM cores,UM NPROCY")

In [ ]:
ppd.loglog_by_group("atm_npes","Atm_Step_4A (AS) speed",
    by=["ocn_npes","atm_cols"], 
    data=data,
    xlabel="UM ranks",
    ylabel="UM speed (d/d)",
    xticks=xticks_npes(data),
    right_axis_fn=pra.plot_right_axis_yd,
    legend_title="MOM cores,UM NPROCY")

In [ ]:
ppd.semilogx_by_group("atm_npes","Atm_Step_4A (AS) * ncpus speed",
    by=["ocn_npes","atm_cols"], 
    data=data,
    xlabel="UM ranks",
    ylabel="UM speed (d/d) per CM2 core",
    xticks=xticks_npes(data),               
    right_yticks=linspace(0,0.01,6),
    right_axis_fn=pra.plot_right_axis_yd,
    legend_title="MOM cores,UM NPROCY")

In [ ]:
series_speed_per_cpu = derive_mean_speed_per_cpu(
    data,
    by=["ocn_npes","atm_cols","atm_npes"])
mean_speed_per_cpu = (
    lambda key:
    list(series_speed_per_cpu[key].items())[0][1])
print("MOM cores UM NPROCY UM Ranks Speed per CM2 core")
for key in series_speed_per_cpu:
    speed = mean_speed_per_cpu(key)
    print(f"{key[0]:-6}     {key[1]:-6}    {key[2]:-6}    {speed:-5.3}")
print("MOM cores: 64 vs 128")
print("UM NPROCY: 12")
print("UM Ranks Speedup")
for npes in [144, 288]:
    print(f"{npes:-6}   {mean_speed_per_cpu((64,12,npes))/mean_speed_per_cpu((128,12,npes)):-5.3}")
print("UM NPROCY: 24")
print("UM Ranks Speedup")
for npes in [288, 576, 1152]:
    print(f"{npes:-6}   {mean_speed_per_cpu((64,24,npes))/mean_speed_per_cpu((128,24,npes)):-5.3}")
print("MOM cores: 128 vs 256")
print("UM NPROCY: 24")
print("UM Ranks Speedup")
for npes in [288, 576, 1152]:
    print(f"{npes:-6}   {mean_speed_per_cpu((128,24,npes))/mean_speed_per_cpu((256,24,npes)):-5.3}")

In [ ]:
npe144=lambda d:d[d["atm_npes"] == 144]

In [ ]:
data=cl(ocn0(omp1(df)))
ppd.loglog_by_group("ncpus","Atm_Step_4A (AS) speed",
    by=["atm_npes"], 
    data=data,
    xlabel="CM2 cores",
    ylabel="UM speed (d/d)",
    xticks=xticks_ncpus(data),             
    right_yticks=[0.5,1,2,5,10],
    right_axis_fn=pra.plot_right_axis_yd,
    legend_title="UM ranks")

In [ ]:
ppd.loglog_by_group("ncpus","Atm_Step_4A (AS) speed",
    by=["npes_per_node"], 
    data=data,
    xlabel="CM2 cores",
    ylabel="UM speed (d/d)",
    xticks=xticks_ncpus(data),             
    right_yticks=[0.5,1,2,5,10],
    right_axis_fn=pra.plot_right_axis_yd,
    legend_title="PEs per node")

In [ ]:
ppd.semilogx_by_group("ncpus","Atm_Step_4A (AS) * ncpus speed",
    by=["npes_per_node"], 
    data=data,
    xlabel="CM2 cores",
    ylabel="UM speed (d/d) per CM2 core",
    xticks=xticks_ncpus(data),
    right_yticks=[0.002,0.005,0.01,0.02],
    right_axis_fn=pra.plot_right_axis_yd,
    legend_title="PEs per node")

In [ ]:
ppd.loglog_by_group("atm_npes","Atm_Step_4A (AS) speed",
    by=["npes_per_node"], 
    data=data,
    xlabel="UM ranks",
    ylabel="UM speed (d/d)",
    xticks=xticks_npes(data),             
    right_yticks=[0.5,1,2,5,10],
    right_axis_fn=pra.plot_right_axis_yd,
    legend_title="PEs per node")

In [ ]:
data=full(omp1(df))

In [ ]:
def report_on_speed(data=full(omp1(df)),speed=None):
    by = ["atm_rows","atm_cols","ocn_rows","ocn_cols","npes_per_node"]
    series_speed = (
        derive_mean_speed(data,by=by)
    if speed is None else
        derive_mean_speed(data,by=by,speed=speed))
    mean_speed = (
        lambda key:
        list(series_speed[key].items())[0][1])
    series_speed_keys = series_speed.keys()
    speed_keys = []
    for key_n in range(4):
        speed_keys.append(sorted(set([key[key_n] for key in series_speed])))
    print("Cascade Lake vs Broadwell")
    print("UM NPROCX UM NPROCY MOM NPROCX NPROCY Speedup")
    for atm_rows in speed_keys[0]:
        for atm_cols in speed_keys[1]:
            for ocn_rows in speed_keys[2]:
                for ocn_cols in speed_keys[3]:
                    key = (atm_rows,atm_cols,ocn_rows,ocn_cols,28)
                    if key in series_speed_keys:
                        print(
                        f"{atm_rows:-5}     {atm_cols:-5}      {ocn_rows:-5}  {ocn_cols:-5}    "
                        f"{mean_speed((atm_rows,atm_cols,ocn_rows,ocn_cols,48))/mean_speed(key):-5.3}")                


In [ ]:
report_on_speed()

In [ ]:
report_on_speed(speed="walltime speed")

In [ ]:
report_on_speed(speed="Ocean speed")

In [ ]:
data=full(ocn0(omp1(df)))

In [ ]:
ppd.loglog_by_group("atm_npes","Atm_Step_4A (AS) speed",
    by=["npes_per_node","atm_cols"], 
    data=data,
    xlabel="UM ranks",
    ylabel="UM speed (d/d)",
    xticks=xticks_npes(data),             
    right_yticks=[0.5,1,2,5,10],
    right_axis_fn=pra.plot_right_axis_yd,
    legend_title="PEs per node,UM NPROCY")

In [ ]:
ppd.semilogx_by_group("atm_npes","Atm_Step_4A (AS) * ncpus speed",
    by=["npes_per_node","atm_cols"], 
    data=data,
    xlabel="UM ranks",
    ylabel="UM speed (d/d) per CM2 core",
    xticks=xticks_npes(data),
    right_yticks=[0.002,0.005,0.01,0.02],
    right_axis_fn=pra.plot_right_axis_yd,
    legend_title="PEs per node,UM NPROCY")

In [ ]:
series_speed = derive_mean_speed(
    data,
    by=["npes_per_node","atm_cols","atm_npes"])
mean_speed = (
    lambda key:
    list(series_speed[key].items())[0][1])
print("Cores per node  UM NPROCY UM Ranks   Speed")
for key in series_speed:
    speed = mean_speed(key)
    print(f"  {key[0]:-6}       {key[1]:-6}     {key[2]:-6}   {speed:-10.3}")
print("Cascade Lake vs Broadwell")
print("UM NPROCY: 12")
print("UM Ranks Speedup")
for npes in [144]:
    print(f"{npes:-6}   {mean_speed((48,12,npes))/mean_speed((28,12,npes)):-5.3}")

In [ ]:
data=full(ocn1(omp1(df)))

In [ ]:
ppd.loglog_by_group("atm_npes","Atm_Step_4A (AS) speed",
    by=["npes_per_node","atm_cols"], 
    data=data,
    xlabel="UM ranks",
    ylabel="UM speed (d/d)",
    xticks=xticks_npes(data),             
    right_yticks=[0.5,1,2,5,10],
    right_axis_fn=pra.plot_right_axis_yd,
    legend_title="PEs per node,UM NPROCY")

In [ ]:
ppd.semilogx_by_group("atm_npes","Atm_Step_4A (AS) * ncpus speed",
    by=["npes_per_node","atm_cols"], 
    data=data,
    xlabel="UM ranks",
    ylabel="UM speed (d/d) per CM2 core",
    xticks=xticks_npes(data),
    right_yticks=[0.002,0.005,0.01,0.02],
    right_axis_fn=pra.plot_right_axis_yd,
    legend_title="PEs per node,UM NPROCY")

In [ ]:
series_speed = derive_mean_speed(
    data,
    by=["npes_per_node","atm_cols","atm_npes"])
mean_speed = (
    lambda key:
    list(series_speed[key].items())[0][1])
print("Cores per node  UM NPROCY UM Ranks   Speed")
for key in series_speed:
    speed = mean_speed(key)
    print(f"  {key[0]:-6}       {key[1]:-6}     {key[2]:-6}   {speed:-10.3}")
print("Cascade Lake vs Broadwell")
print("UM NPROCY: 12")
print("UM Ranks Speedup")
for npes in [144]:
    print(f"{npes:-6}   {mean_speed((48,12,npes))/mean_speed((28,12,npes)):-5.3}")
print("UM NPROCY: 24")
print("UM Ranks Speedup")
for npes in [288]:
    print(f"{npes:-6}   {mean_speed((48,24,npes))/mean_speed((28,24,npes)):-5.3}")
print("UM NPROCY: 28")
print("UM Ranks Speedup")
for npes in [672]:
    print(f"{npes:-6}   {mean_speed((48,28,npes))/mean_speed((28,28,npes)):-5.3}")    

In [ ]:
data=full(omp1(ocn1(atm2(df))))
ppd.loglog_by_group("ncpus","Atm_Step_4A (AS) speed",
    by=["npes_per_node","atm_cols","ocn_npes"], 
    data=data,
    xlabel="ACCESS-CM2 cores",
    ylabel="UM speed (d/d)",
    xticks=xticks_ncpus(data),
    yticks=[2500,3000,3500],                
    right_yticks=[2,5,8,10,20],
    right_axis_fn=pra.plot_right_axis_yd,
    legend_title="PEs per node, UM NPROCY, MOM cores")

In [ ]:
data=full(omp1(ocn1(atm2(df))))
ppd.semilogx_by_group("ncpus","Atm_Step_4A (AS) * ncpus speed",
    by=["npes_per_node","atm_cols","ocn_npes"], 
    data=data,
    xlabel="ACCESS-CM2 cores",
    ylabel="UM speed (d/d) per CM2 core",
    xticks=xticks_ncpus(data),
    right_yticks=[0.002,0.005,0.01,0.02],
    right_axis_fn=pra.plot_right_axis_yd,
    legend_title="PEs per node, UM NPROCY, MOM cores")

In [ ]:
import pandas as pd
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

In [ ]:
#df.sort_values(["omp_num_threads","ocn_npes","atm_npes","atm_cols"])

In [ ]:
df.shape

In [ ]:
bw(df).shape

In [ ]:
#bw(df).sort_values(["omp_num_threads","ocn_npes","atm_npes","atm_cols"])

In [ ]:
df.loc[1,:]